In [1]:
pip install rhinoMorph

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install JPype1

In [1]:
import pandas as pd
import rhinoMorph
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score

In [2]:
df1 = pd.read_csv("jmj_oliveyoung1_new.csv")
df2 = pd.read_csv("jmj_oliveyoung2_new.csv")
df3 = pd.read_csv("jmj_oliveyoung3_new.csv")
df4 = pd.read_csv("jmj_oliveyoung4_new.csv")
df5 = pd.read_csv("jmj_oliveyoung5_new.csv")
df6 = pd.read_csv("jmj_oliveyoung6_new.csv")

In [3]:
df5

,Unnamed: 0,id,rating,reviews
0,21,dlekqls****,5,진짜진짜좋아요 킁카킁카 계속 맡게되요 ㅠㅠ 지속력도 좋아요
1,23,wlgp****,5,최애 향수 ㅠㅠ 친구가 생일선물로 상품권 줘서 제돈 조굼 보태서 바로 이거 사쏘요 ...
2,26,amocr****,4,향 되게 좋아여 지속력도 오래가서 학교 끝나고 나서도 친구가 향수 바꿨냐고 물어봄
3,28,k202307136****,5,여자들이 쓰기에 괜찮은 제품이에요전체적으로 엄청 달달한 향입니다여자친구도 굉장히 좋...
4,30,mimisis01,5,"재구매 의사 있고 속옷,수건,베개,이불 등등 뿌려두면 은은한 향으로 기분도 좋아집니다."
...,...,...,...,...
2873,5902,ctv****,5,향은 진짜 너무 좋은데. 새벽 갬성이나 약간 옛날 갬성 느낌도 나고.. 지속력은 다...
2874,5903,jinsol****,4,향이 좋고 보관이 간편해 보여서 구매했는데 지속력은 나쁘지 않고 향은 역시 좋아요!...
2875,5904,tiger****,5,병에 담긴거 사고싶었는데 아쉽게도 재고가 없어서튜브 형식으로 된거 샀어요 향은 너무...
2876,5905,ksh67****,5,딸이 바르고 다니는거 냄새가 좋아서 바로 어디에서 샀는지 물어보고 구매했어요~^^ ...


In [4]:
# 데이터프레임 하나로 병합
df_all = pd.concat([df1, df2, df3, df4, df5, df6])

In [5]:
df_all = df_all.drop(columns='Unnamed: 0', axis=1)

In [6]:
df_all

,id,rating,reviews
0,크림애비,5,수부지 다 모여라!!걱정했던만큼 꾸덕한 제형이 아니라 너무 마음에 듭니다 수분감은 ...
1,다블리,5,🩵 에스트라 아토베리어 365 크림 한달 사용 리뷰 입니다! 🩵☑️ 이 제품을 계속...
2,스누비,5,캡슐이 눈에 보이기도 하고 확실히 보습에 좋은 제품 같아요겨울철이라 보습 꼼꼼하게 ...
3,진예리,5,겨울철에는 항상 찾게 되는 크림이에요 캡슐이 들어있어서 수분감이 더 풍부한 느낌
4,두부후추메주,5,재구매템이고 떨어져가면 바로 쟁여요 제 피부를 책임져주는 아이…
...,...,...,...
2542,졔졔니,5,동생이 사달라고해서 구매했어요!좋은것 같아요~ 한달 사용 잘하고 리뷰하라고 할게요ㅎㅎ
2543,전설의올영,4,자연스러운 볼륨감과 에센스를 위해 구매하였는데요 향도 좋고 끈적임도 없네요! 추천...
2544,운수대톨,5,다슈 데일리 볼륨업 컬크림 자연스러운 헤어 스타일링에 도움이 됩니다
2545,빠밤,4,"남편이 사용한다고 구매했는데, 편하게 데일리로 좋다고해요."


In [7]:
# 리뷰에 따라 긍부정 라벨링, 함수
def label_review(score):
    if score <= 3:
        return 0
    else:
        return 1

In [8]:
# 라벨링 적용
df_all['label'] = df_all['rating'].apply(label_review)

In [9]:
df_all

,id,rating,reviews,label
0,크림애비,5,수부지 다 모여라!!걱정했던만큼 꾸덕한 제형이 아니라 너무 마음에 듭니다 수분감은 ...,1
1,다블리,5,🩵 에스트라 아토베리어 365 크림 한달 사용 리뷰 입니다! 🩵☑️ 이 제품을 계속...,1
2,스누비,5,캡슐이 눈에 보이기도 하고 확실히 보습에 좋은 제품 같아요겨울철이라 보습 꼼꼼하게 ...,1
3,진예리,5,겨울철에는 항상 찾게 되는 크림이에요 캡슐이 들어있어서 수분감이 더 풍부한 느낌,1
4,두부후추메주,5,재구매템이고 떨어져가면 바로 쟁여요 제 피부를 책임져주는 아이…,1
...,...,...,...,...
2542,졔졔니,5,동생이 사달라고해서 구매했어요!좋은것 같아요~ 한달 사용 잘하고 리뷰하라고 할게요ㅎㅎ,1
2543,전설의올영,4,자연스러운 볼륨감과 에센스를 위해 구매하였는데요 향도 좋고 끈적임도 없네요! 추천...,1
2544,운수대톨,5,다슈 데일리 볼륨업 컬크림 자연스러운 헤어 스타일링에 도움이 됩니다,1
2545,빠밤,4,"남편이 사용한다고 구매했는데, 편하게 데일리로 좋다고해요.",1


In [10]:
# 라벨링 결과 확인,
# 긍정 데이터 약 95%, 부정 데이터 약 5%
df_all['label'].value_counts()

1    16510
0      849
Name: label, dtype: int64

In [15]:
16510/17359

0.9510916527449738

In [16]:
# 라이노 형태소 분석기 초기화
rhn = rhinoMorph.startRhino()

# 형태소 분석을 위한 전처리 함수
def preprocess_review(review, rhn):
    # 텍스트 정제, 한글 문자와 공백을 제외한 모든 문자
    cleaned_review = re.sub(r'[^가-힣\s]', '', review)
    
    # 형태소 분석, 필요한 품사(명사, 동사, 형용사)만 추출
    morphs = rhinoMorph.onlyMorph_list(rhn, cleaned_review, pos=['NNG', 'NNP', 'VV', 'VA'], eomi=True)
    return ' '.join(morphs)

filepath:  C:\Users\jinmi\anaconda3\Lib\site-packages
classpath:  C:\Users\jinmi\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!


In [17]:
# 'review' 컬럼에 형태소 분석 적용
df_all['processed_review'] = df_all['reviews'].apply(lambda x: preprocess_review(x, rhn))

In [18]:
df_all

,id,rating,reviews,label,processed_review
0,크림애비,5,수부지 다 모여라!!걱정했던만큼 꾸덕한 제형이 아니라 너무 마음에 듭니다 수분감은 ...,1,수부 모이다 걱정 하다 꾸다 덕 하다 제형 마음 들다 수분 감다 사용 하다 제품 세...
1,다블리,5,🩵 에스트라 아토베리어 365 크림 한달 사용 리뷰 입니다! 🩵☑️ 이 제품을 계속...,1,에스 트다 토 버리다 크림 사용 리뷰 제품 구매 하다 이유 복합 피부 에스 트다 토...
2,스누비,5,캡슐이 눈에 보이기도 하고 확실히 보습에 좋은 제품 같아요겨울철이라 보습 꼼꼼하게 ...,1,캡슐 눈 보이다 하다 보습 좋다 제품 같다 겨울철 보습
3,진예리,5,겨울철에는 항상 찾게 되는 크림이에요 캡슐이 들어있어서 수분감이 더 풍부한 느낌,1,겨울철 찾다 되다 크림 캡슐 들다 있다 수분 감 느낌
4,두부후추메주,5,재구매템이고 떨어져가면 바로 쟁여요 제 피부를 책임져주는 아이…,1,구매 떨어지다 쟁이다 피부 책임지다
...,...,...,...,...,...
2542,졔졔니,5,동생이 사달라고해서 구매했어요!좋은것 같아요~ 한달 사용 잘하고 리뷰하라고 할게요ㅎㅎ,1,동생 사다 달라다 좋다 같다 사용 잘하다 리뷰 하다
2543,전설의올영,4,자연스러운 볼륨감과 에센스를 위해 구매하였는데요 향도 좋고 끈적임도 없네요! 추천...,1,자연 볼륨감 에센스 위하다 향도 좋다 끈적이다 없다
2544,운수대톨,5,다슈 데일리 볼륨업 컬크림 자연스러운 헤어 스타일링에 도움이 됩니다,1,데일리 볼륨 업다 컬 크림 자연 헤어 스타일 링 도움 되다
2545,빠밤,4,"남편이 사용한다고 구매했는데, 편하게 데일리로 좋다고해요.",1,남편 편하다 데일리 좋다 고해


In [19]:
# 불용어
my_stop_words = ['아','휴','아이구','아이쿠','아이고','어','나','우리','저희','따라','의해','을','를','에',
                     '의','가','으로','로','에게','뿐이다','의거하여','근거하여','입각하여','기준으로','예하면',
                     '예를 들면','예를 들자면','저','소인','소생','저희','지말고','하지마','하지마라','다른','물론',
                     '또한','그리고','비길수 없다','해서는 안된다','뿐만 아니라','만이 아니다','만은 아니다',
                     '막론하고','관계없이','그치지 않다','그러나','그런데','하지만','든간에','논하지 않다',
                     '따지지 않다','설사','비록','더라도','아니면','만 못하다','하는 편이 낫다','불문하고','향하여',
                     '향해서','향하다','쪽으로','틈타','이용하여','타다','오르다','제외하고','이 외에','이 밖에',
                     '하여야','비로소','한다면 몰라도','외에도','이곳','여기','부터','기점으로','따라서',
                     '할 생각이다','하려고하다','이리하여','그리하여','그렇게 함으로써','하지만','일때','할때',
                     '앞에서','중에서','보는데서','으로써','로써','까지','해야한다','일것이다','반드시','할줄알다',
                     '할수있다','할수있어','임에 틀림없다','한다면','등','등등','제','겨우','단지','다만','할뿐',
                     '딩동','댕그','대해서','대하여','대하면','훨씬','얼마나','얼마만큼','얼마큼','남짓','여',
                     '얼마간','약간','다소','좀','조금','다수','몇','얼마','지만','하물며','또한','그러나',
                     '그렇지만','하지만','이외에도','대해 말하자면','뿐이다','다음에','반대로','반대로 말하자면',
                     '이와 반대로','바꾸어서 말하면','바꾸어서 한다면','만약','그렇지않으면','까악','툭','딱',
                     '삐걱거리다','보드득','비걱거리다','꽈당','응당','해야한다','에 가서','각','각각','여러분',
                     '각종','각자','제각기','하도록하다','와','과','그러므로','그래서','고로','한 까닭에',
                     '하기 때문에','거니와','이지만','대하여','관하여','관한','과연','실로','아니나다를가',
                     '생각한대로','진짜로','한적이있다','하곤하였다','하','하하','허허','아하','거바','와','오',
                     '왜','어째서','무엇때문에','어찌','하겠는가','무슨','어디','어느곳','더군다나','하물며',
                     '더욱이는','어느때','언제','야','이봐','어이','여보시오','흐흐','흥','휴','헉헉','헐떡헐떡',
                     '영차','여차','어기여차','끙끙','아야','앗','아야','콸콸','졸졸','좍좍','뚝뚝','주룩주룩',
                     '솨','우르르','그래도','또','그리고','바꾸어말하면','바꾸어말하자면','혹은','혹시','답다',
                     '및','그에 따르는','때가 되어','즉','지든지','설령','가령','하더라도','할지라도','일지라도',
                     '지든지','몇','거의','하마터면','인젠','이젠','된바에야','된이상','만큼\t어찌됏든','그위에',
                     '게다가','점에서 보아','비추어 보아','고려하면','하게될것이다','일것이다','비교적','좀',
                     '보다더','비하면','시키다','하게하다','할만하다','의해서','연이서','이어서','잇따라','뒤따라',
                     '뒤이어','결국','의지하여','기대여','통하여','자마자','더욱더','불구하고','얼마든지',
                     '마음대로','주저하지 않고','곧','즉시','바로','당장','하자마자','밖에 안된다','하면된다',
                     '그래','그렇지','요컨대','다시 말하자면','바꿔 말하면','즉','구체적으로','말하자면',
                     '시작하여','시초에','이상','허','헉','허걱','바와같이','해도좋다','해도된다','게다가',
                     '더구나','하물며','와르르','팍','퍽','펄렁','동안','이래','하고있었다','이었다','에서',
                     '로부터','까지','예하면','했어요','해요','함께','같이','더불어','마저','마저도','양자',
                     '모두','습니다','가까스로','하려고하다','즈음하여','다른','다른 방면으로','해봐요','습니까',
                     '했어요','말할것도 없고','무릎쓰고','개의치않고','하는것만 못하다','하는것이 낫다','매',
                     '매번','들','모','어느것','어느','로써','갖고말하자면','어디','어느쪽','어느것','어느해',
                     '어느 년도','라 해도','언젠가','어떤것','어느것','저기','저쪽','저것','그때','그럼','그러면',
                     '요만한걸','그래','그때','저것만큼','그저','이르기까지','할 줄 안다','할 힘이 있다','너',
                     '너희','당신','어찌','설마','차라리','할지언정','할지라도','할망정','할지언정','구토하다',
                     '게우다','토하다','메쓰겁다','옆사람','퉤','쳇','의거하여','근거하여','의해','따라','힘입어',
                     '그','다음','버금','두번째로','기타','첫번째로','나머지는','그중에서','견지에서','형식으로 쓰여',
                 '입장에서','위해서','단지','의해되다','하도록시키다','뿐만아니라','반대로','전후','전자','앞의것','잠시','잠깐','하면서','그렇지만','다음에','그러한즉','그런즉','남들','아무거나','어찌하든지','같다','비슷하다','예컨대','이럴정도로','어떻게','만약','만일','위에서 서술한바와같이','인 듯하다','하지 않는다면','만약에','무엇','무슨','어느','어떤','아래윗','조차','한데','그럼에도 불구하고','여전히','심지어','까지도','조차도','하지 않도록','않기 위하여','때','시각','무렵','시간','동안','어때','어떠한','하여금','네','예','우선','누구','누가 알겠는가','아무도','줄은모른다','줄은 몰랏다','하는 김에','겸사겸사','하는바','그런 까닭에','한 이유는','그러니','그러니까','때문에','그','너희','그들','너희들','타인','것','것들','너','위하여','공동으로','동시에','하기 위하여','어찌하여','무엇때문에','붕붕','윙윙','나','우리','엉엉','휘익','윙윙','오호','아하','어쨋든','만 못하다\t하기보다는','차라리','하는 편이 낫다','흐흐','놀라다','상대적으로 말하자면','마치','아니라면','쉿','그렇지 않으면','그렇지 않다면','안 그러면','아니었다면','하든지','아니면','이라면','좋아','알았어','하는것도','그만이다','어쩔수 없다','하나','일','일반적으로','일단','한켠으로는','오자마자','이렇게되면','이와같다면','전부','한마디','한항목','근거로','하기에','아울러','하지 않도록','않기 위해서','이르기까지','이 되다','로 인하여','까닭으로','이유만으로','이로 인하여','그래서','이 때문에','그러므로','그런 까닭에','알 수 있다','결론을 낼 수 있다','으로 인하여','있다','어떤것','관계가 있다','관련이 있다','연관되다','어떤것들','에 대해','이리하여','그리하여','여부','하기보다는','하느니','하면 할수록','운운','이러이러하다','하구나','하도다','다시말하면','다음으로','에 있다','에 달려 있다','우리','우리들','오히려','하기는한데','어떻게','어떻해','어찌됏어','어때','어째서','본대로','자','이','이쪽','여기','이것','이번','이렇게말하자면','이런','이러한','이와 같은','요만큼','요만한 것','얼마 안 되는 것','이만큼','이 정도의','이렇게 많은 것','이와 같다','이때','이렇구나','것과 같이','끼익','삐걱','따위','와 같은 사람들','부류의 사람들','왜냐하면','중의하나','오직','오로지','에 한하다','하기만 하면','도착하다','까지 미치다','도달하다','정도에 이르다','할 지경이다','결과에 이르다','관해서는','여러분','하고 있다','한 후','혼자','자기','자기집','자신','우에 종합한것과같이','총적으로 보면','총적으로 말하면','총적으로','대로 하다','으로서','참','그만이다','할 따름이다','쿵','탕탕','쾅쾅','둥둥','봐','봐라','아이야','아니','와아','응','아이','참나','년','월','일','령','영','일','이','삼','사','오','육','륙','칠','팔','구','이천육','이천칠','이천팔','이천구','하나','둘','셋','넷','다섯','여섯','일곱','여덟','아홉','령','영']

In [20]:
len(my_stop_words)

675

In [21]:
# TF-IDF 벡터화, my_stop_words 불용어 적용
tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words)
X = tfidf_vectorizer.fit_transform(df_all['processed_review'])

C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '만큼', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '어찌됏든', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(


In [27]:
# 고유 단어 목록 추출
feature_names = tfidf_vectorizer.get_feature_names_out()

# DataFrame으로 변환
df_tfidf = pd.DataFrame(X.toarray(), columns=feature_names)

In [28]:
df_tfidf

,가거,가게,가격,가격대,가격차이,가경,가공식품,가구,가까워다,가까이,...,힌두,힌트,힐러,힐링,힘겹다,힘들다,힘들이다,힘쓰다,힘없다,힘주다
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# 원본 리뷰 텍스트와 라벨을 함께 분리
from sklearn.model_selection import train_test_split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    df_all['processed_review'], df_all['label'], test_size=0.2, random_state=42)

In [24]:
# TF-IDF 벡터화 (훈련 데이터에만 fit_transform 적용)
X_train = tfidf_vectorizer.fit_transform(X_train_texts)
X_test = tfidf_vectorizer.transform(X_test_texts)

In [26]:
# 교차 검증을 위한 파라미터 범위 지정
import numpy as np
alphas = np.logspace(-6, 6, 13)

## 모델링 Start(릿지, 라쏘, 엘라스틱넷)

In [28]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [29]:
# 릿지 회귀
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
        cv=5)

In [54]:
# 릿지모델 저장 코드
with open('ridge_cv.pkl', 'wb') as fw:
    pickle.dump(ridge_cv, fw)

In [55]:
# Load the saved RidgeCV model(저장된 모델들 불러오는 코드)
with open('ridge_cv.pkl', 'rb') as fr:
    loaded_ridge_cv = pickle.load(fr)

In [30]:
# 라쏘 회귀
lasso_cv = LassoCV(alphas=alphas, cv=5)
lasso_cv.fit(X_train, y_train)

C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.10235653170337855, tolerance: 0.051013844630479754
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1399105827290441, tolerance: 0.052276982626699046
  model = cd_fast.sparse_enet_coordinate_descent(


LassoCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
        cv=5)

In [56]:
# 라쏘모델 저장 코드
with open('lasso_cv.pkl', 'wb') as fw:
    pickle.dump(ridge_cv, fw)

In [57]:
# 라쏘( 불러오는 코드)
with open('lasso_cv.pkl', 'rb') as fr:
    loaded_lasso_cv = pickle.load(fr)

In [31]:
# 엘라스틱넷
elastic_cv = ElasticNetCV(alphas=alphas, cv=5)
elastic_cv.fit(X_train, y_train)

C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11558633141808627, tolerance: 0.052276982626699046
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.09104913264209813, tolerance: 0.05137533753375336
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\jinmi\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:592: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06385906764427318, tolerance: 0.05182650765076509
  model = cd_fast.sparse_enet_coordinate_descent(


ElasticNetCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
             cv=5)

In [58]:
# 라쏘모델 저장 코드
with open('elastic_cv.pkl', 'wb') as fw:
    pickle.dump(elastic_cv, fw)

In [59]:
# 엘라스틱넷( 불러오는 코드)
with open('lasso_cv.pkl', 'rb') as fr:
    loaded_elastic_cv = pickle.load(fr)

In [34]:
# TF-IDF 벡터화 과정에서 생성된 고유 단어 목록을 추출
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_names

array(['가거', '가게', '가격', ..., '힘쓰다', '힘없다', '힘주다'], dtype=object)

In [35]:
# 감정 점수 계산 함수
def calculate_sentiment_score(model_coefficients, feature_names, text):
    score = 0
    words = text.split()
    for word in words:
        if word in feature_names:
            index = np.where(feature_names == word)[0][0]
            score += model_coefficients[index]
    return score

In [36]:
# 모델 성능 평가 함수
def evaluate_model_sentiment(model, X_texts, y, feature_names):
    test_scores = [calculate_sentiment_score(model.coef_, feature_names, text) 
                   for text in X_texts]
    predicted_labels = [1 if score > 0 else 0 for score in test_scores]
    accuracy = accuracy_score(y, predicted_labels)
    return accuracy

## 모델들 성능 검증 평가

In [37]:
# 릿지 모델 평가
ridge_test_accuracy = evaluate_model_sentiment(ridge_cv, X_test_texts, y_test, feature_names)
print(f"릿지 정확도:{ridge_test_accuracy}")

릿지 정확도:0.8101958525345622


In [40]:
# 라쏘 모델 평가
lasso_test_accuracy = evaluate_model_sentiment(lasso_cv, X_test_texts, y_test, feature_names)
print(f"라쏘 정확도:{lasso_test_accuracy}")

라쏘 정확도:0.7548963133640553


In [39]:
# 엘라스틱넷 모델 평가
elastic_test_accuracy = evaluate_model_sentiment(elastic_cv, X_test_texts, y_test, feature_names)
print(f"엘라스틱넷 정확도: {elastic_test_accuracy}")

엘라스틱넷 정확도: 0.7523041474654378


## 모델별 감성사전

In [42]:
# 감정사전 추출 함수(회귀계수>0: 긍정사전에 추가, 회귀계수<0: 부정사전에 추가)
def extract_sentiment_dictionary(model, feature_names):
    coefficients = model.coef_
    positive_words = feature_names[coefficients > 0]
    negative_words = feature_names[coefficients < 0]
    return positive_words, negative_words

# 1. 릿지 모델 감성사전

In [43]:
# 릿지 모델 감정사전 추출
positive_words_ridge, negative_words_ridge = extract_sentiment_dictionary(ridge_cv, feature_names)
print("릿지 회귀 - 긍정:", positive_words_ridge)
print("릿지 회귀 - 부정:", negative_words_ridge)

릿지 회귀 - 긍정: ['가거' '가게' '가격' ... '힘쓰다' '힘없다' '힘주다']
릿지 회귀 - 부정: ['가까워다' '가까이' '가깝다' ... '흰색' '히아신스' '히피']


In [65]:
print(*negative_words_ridge)

가까워다 가까이 가깝다 가늘다 가다 가두다 가라앉히다 가량 가려보다 가렵다 가뭄 가사 가약 가오 가요 가장 가정 가지 가짜 간만 간식 간이 간조 간지 간지럽다 간채 갈고다 갈리다 갈색빛 갈아타다 갈치 감당 감미 감사 감수 감염 감저 감쪽같다 감호 강도 강아지 강의 강점 강제 강하 갖다 갖추다 같아지다 개기름 개똥 개리 개별 개봉 개선 개성적 개인 개인적 개인차 개작 거도 거란 거리 거리다 거여 거울 건강 건강식품 건드리다 건성 건지다 걸다 걸레 걸리다 걸치다 검수 검은색 겉면 겨울비 격일 결과 결론 결막염 결부 결제 결함 결혼기념일 겸용 겹치다 경우 고갈 고객 고객센터 고건 고깃집 고노 고다 고디 고려 고민중 고발 고보 고생 고속 고손 고어 고올 고요아침 고유 고장 고전 고정 고제 고집 고쳐다 고하다 고한 고호 곰팡이 곱다 곱창 공간 공갈 공기 공들이다 공장 공장부 과거 과다 과대광고 과대평가 과로 과언 과장 과전 과하다 관계자 관리하다 관상 관심 관자놀이 광고 광고인 광대 광채 광화 괜찮다 교육 교환 교훈 구기다 구독자 구리다 구립 구버 구분 구석 구처 구하다 국내외 굴곡 굴리다 굵기 궁금하다 권하다 권하진 귀신 귀찬다 균형 그라다 그러다 그러하다 그렇다 그레이스 그루 그리다 그리하다 그림자 그을다 그을리다 그을음 그전 그지 극성 극소량 극적 극지 극찬 근래 근처 글로우 글자 긁다 긁히다 금방 금식 급하다 기다 기대 기대다 기도 기름 기름때 기름종이 기름지다 기본 기분 기사 기수 기술 기시다 기업 기자 기재 기저 기존 기한 기회 기획력 긴급 긷다 길다 길들이다 길이 길지 까다 까다롭다 깎이다 깔때 깔리다 깨다 깨지다 꺼내다 껍질 꼬마 꼽다 꾸미다 끈덕지다 끌다 끌레 끝내다 끝물 끼다 끼우다 끼이다 나나 나누다 나다 나들이 나머지 나무 나무껍질 나쁘다 나시 나신 나아지다 나오다 나위 나진 낚다 낚이다 난가 난관 난리 난장판 날뛰다 날라가다 날림 날수 날아가다 날아다니다 날짜 남고 남구 남기다 남길수 남다르다 남모르다 남복 남서 남성 남성적 남아나다 남자애 남친

In [44]:
# 릿지모델의 긍정, 부정 단어 갯수
print(len(positive_words_ridge),len(negative_words_ridge))

6848 2050


In [ ]:
########################여가까지 릿지모델임 #########################################

# 2. 라쏘 모델 감성사전

In [47]:
positive_words_lasso, negative_words_lasso = extract_sentiment_dictionary(lasso_cv, feature_names)
print("라쏘 회귀 - 긍정 사전:", positive_words_lasso)
print("라쏘 회귀 - 부정 사전:", negative_words_lasso)

라쏘 회귀 - 긍정 사전: ['각질' '거품' '겨울' '고민' '끈적이다' '남편' '데일리' '되다' '두피' '들다' '뛰어나다' '마스카라' '마음'
 '만족' '믿다' '바디로션' '발라' '보습력' '부드럽다' '빠르다' '뿌리다' '샴푸' '선물' '세럼' '세일' '순하다'
 '쉽다' '쓰기' '쓰다' '쓸다' '어울리다' '에디션' '예쁘다' '오일' '올인' '완전' '이제' '자극' '자연' '자주'
 '잡다' '쟁이다' '제품' '좋다' '좋아하다' '주다' '중성적' '찾다' '최고' '추천' '친구' '크림' '타입' '통째'
 '편하다' '할인' '향도' '향수']
라쏘 회귀 - 부정 사전: ['가다' '건성' '광고' '그렇다' '그리하다' '기대' '기대다' '기름' '기름지다' '깨지다' '난리' '느낌' '다리다'
 '닦다' '돌다' '돌아가다' '두껍다' '뚜껑' '뜯다' '리뷰' '많다' '맞다' '모공' '모르다' '못하다' '문의'
 '바뀌다' '배송' '백화점' '복숭아' '부러지다' '불다' '비추다' '비하다' '사다' '사라지다' '산거' '살다' '새다'
 '색감' '생각' '생각하다' '실제' '심하다' '쏘다' '아깝다' '아쉽다' '안되다' '알다' '어떻하다' '어렵다' '얼굴'
 '없다' '연하다' '오가다' '올라오다' '자체' '잘못' '정도' '좁쌀' '증정품' '지나다' '지성' '지우다' '짜증'
 '짧다' '찍히다' '취향' '케이스' '쿠션' '표현' '해소' '혹하다' '홍조' '화장지' '효과' '후기']


In [63]:
#라쏘모델의 긍정, 부정 단어 갯수
print(len(positive_words_ridge),len(negative_words_ridge))

6848 2050


In [ ]:
######################################## 여기까지가 라쏘 모델임 ############################################

## 엘라스틱넷 모델 감성사전

In [49]:
positive_words_elastic, negative_words_elastic = extract_sentiment_dictionary(elastic_cv, feature_names)
print("엘라스틱넷 - 긍정 사전:", positive_words_elastic)
print("엘라스틱넷 - 부정 사전:", negative_words_elastic)

엘라스틱넷 - 긍정 사전: ['가격' '가지다' '각질' '감다' '강추' '거품' '건조' '겨울' '겨울철' '계속' '계절' '고르다' '고민' '고민하'
 '관리' '구성' '기획' '꾸다' '끈적이다' '나오다' '날씨' '남다' '남편' '달달' '대용량' '데일리' '되다'
 '두피' '들다' '들어가다' '디다' '뛰어나다' '라이트' '레드' '레이' '로에' '루나' '마스카라' '마스크' '마음'
 '만족' '매장' '먹다' '모로칸' '모으다' '미니' '믿다' '바닐라' '바디로션' '바르다' '바틀' '발라' '보습력'
 '부드럽다' '브라운' '비싸다' '빌리' '빠르다' '뿌리다' '사계절' '사이즈' '샴푸' '서클' '선물' '세럼' '세일'
 '세트' '속눈썹' '수분' '순하다' '쉽다' '스크럽' '스트레스' '써보다' '쓰기' '쓰다' '쓰리다' '쓸다' '아일'
 '아침' '얇다' '양조' '어울리다' '어쩌다' '에디션' '예쁘다' '오다' '오래가다' '오일' '올리다' '올인' '완전'
 '용량' '유명하다' '유목민' '이쁘다' '이유' '이제' '자극' '자다' '자연' '자주' '작다' '잡다' '쟁이다'
 '적당' '전체적' '정착' '제품' '좋다' '좋아하다' '주다' '중성적' '지속력' '지인' '진정효' '질레트' '짜다'
 '차이' '찾다' '챙기다' '최고' '추천' '친구' '컬러' '쿠폰' '크다' '크림' '타입' '탈모' '테스터' '토너'
 '통째' '트다' '파우더' '파우치' '팔레트' '편하다' '피부' '피지' '필수' '할인' '해보다' '행사' '향도'
 '향수' '향인' '호불호']
엘라스틱넷 - 부정 사전: ['가다' '가오' '간지럽다' '감당' '개봉' '개인적' '거리다' '건성' '건지다' '고정' '공간' '광고' '광채'
 '괜찮다' '교환' '궁금하다' '그렇다' '그리하다' '글로우' '기대' '기대다' '기름' '기름지다' '기재' '깔리다'
 '깨지다' 

In [60]:
#엘라스틱넷모델의 긍정, 부정 단어 갯수
print(len(positive_words_ridge),len(negative_words_ridge))

6848 2050


In [ ]:
####################################################끝#######################################################

In [29]:
# 회귀분석 모델 생성 및 학습
model = LogisticRegression()
model.fit(X, df_all['label'])

LogisticRegression()

In [30]:
# 감성 사전 구축
feature_names = tfidf_vectorizer.get_feature_names_out()
coefficients = model.coef_.flatten()

In [31]:
# 회귀계수에 따른 단어 감성 판별
positive_words = feature_names[coefficients > 0]
negative_words = feature_names[coefficients < 0]

In [32]:
# 결과 출력
print("긍정 단어:", positive_words)
print("부정 단어:", negative_words)

print("긍정 단어 수치:", len(positive_words))
print("부정 단어 수치:", len(negative_words))

긍정 단어: ['가거' '가게' '가격' ... '힘쓰다' '힘없다' '힘주다']
부정 단어: ['가까워다' '가까이' '가깝다' ... '흰색' '히피' '힘들이다']
긍정 단어 수치: 7671
부정 단어 수치: 1994


In [47]:
print(*negative_words)

가까워다 가까이 가깝다 가다 가두다 가량 가려보다 가바 가약 가오 가요 가정 가지 가짜 가치 각인 각화증 간만 간이 간조 간지 간지럽다 간채 갈고다 갈리다 갈색 갈아타다 갈치 감당 감사 감수 감저 강도 강아지 강점 강제 강하 강하다 갖다 갖추다 개기름 개똥 개리 개별 개봉 개선 개성적 개이득 개인 개인적 개인차 개작 거근 거란 거리 거리다 거여 거울 거짓말 거칠다 건강 건드리다 건성 건지다 걸다 걸레 걸리다 걸치다 검다 검수 검은색 겉돌다 겉면 격일 결과 결론 결론적 결막염 결정 결제 결함 겸용 겹치다 경우 계산하다 계열 고객 고객센터 고건 고깃집 고다 고려 고민중 고밀 고발 고생 고속 고어 고올 고유 고장 고전 고정 고제 고집 고쳐다 고하다 고호 곰팡이 곱다 공간 공갈 공기 공동 공들이다 공장 공장부 과거 과대광고 과대평가 과장 과전 과정 과하다 관계자 관리하다 관상 관심 관자놀이 광고 광고인 광채 광화 괜찮다 괴롭다 교육 교환 교훈 구리다 구린내 구립 구매동기 구버 구분 구석 구처 구하다 국가 군용 굴리다 굵기 궁금하다 권하다 권하진 귀신 귀차니즘 귀찬다 그날그날 그러다 그러하다 그렇다 그레이스 그루 그리다 그리하다 그림자 그을다 그을리다 그지 극소량 극적 극지 극찬 근거 근래 근처 글로우 글자 긁다 긁히다 금방 금식 금액 급하다 기간 기다 기다리다 기대 기대다 기도 기랑 기름 기름때 기름종이 기름지다 기본 기분 기사 기술 기시다 기억 기용 기재 기전 기존 기한 기회 기획력 긴급 긴하다 길다 길들이다 길지 까다롭다 깎이다 깔다 깔리다 깨다 깨지다 꺼내다 껍질 꾸미다 끌다 끝내다 끝물 끼가다 끼다 끼우다 끼이다 나나 나누다 나다 나라 나머지 나무 나무껍질 나봐 나시 나아지다 나오다 나위 나중 나진 낚다 낚이다 난리 난장판 날뛰다 날라가다 날리다 날림 날아가다 날아다니다 날짜 남고 남기다 남다르다 남리 남모르다 남복 남아나다 남자애 남친 남평 낭군 낭낭 낭비 낭패 낮다 낮추다 내가다 내년 내놓다 내다 내리다 내신 내역 내용물 내장 내지 내츄럴 내친김 냄새 냄새

In [48]:
import numpy as np
# 모델 지정 및 파라미터 범위 설정
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}
param_range = np.logspace(-3, 3, 7) # 파라미터 범위지정

# 교차 검증을 이용해 최적 파라미터 찾기
best_params = {}
for name, model in models.items():
    best_score = -np.inf
    for param in param_range:
        model.set_params(alpha=param)
        scores = cross_val_score(model, X, y, cv=5)
        if np.mean(scores) > best_score:
            best_score = np.mean(scores)
            best_params[name] = param

# 최적 파라미터로 모델 학습 및 회귀 계수 분석
for name, model in models.items():
    model.set_params(alpha=best_params[name])
    model.fit(X, y)
    # 회귀 계수
    coefficients = model.coef_
    # 단어 목록
    feature_names = tfidf_vectorizer.get_feature_names_out()
    # 긍정 및 부정 사전 구축
    positive_words = feature_names[coefficients > 0]
    negative_words = feature_names[coefficients < 0]
    print(f"{name} 회귀 - 긍정 사전: {positive_words}, 부정 사전: {negative_words}")


NameError: name 'y' is not defined

In [91]:
models[0]

KeyError: 0